In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### 読み込み

In [ ]:
df_all = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2020/train.csv")

In [ ]:
df_all.head()

In [ ]:
df_all.columns

## データ確認

In [ ]:
df_all[['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A',
        'Dis', 'Orientation', 'Dir', 'NflId', 'DisplayName', 'JerseyNumber']].head()

In [ ]:
df_all[['Season', 'YardLine', 'Quarter', 'GameClock', 
        'PossessionTeam', 'Down', 'Distance', 'FieldPosition']].head()

In [ ]:
df_all[['HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'NflIdRusher', 'OffenseFormation', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel']].head()

In [ ]:
df_all[['PlayDirection', 'TimeHandoff','TimeSnap', 'Yards']].head()

In [ ]:
df_all[['PlayerHeight', 'PlayerWeight', 'PlayerBirthDate',
       'PlayerCollegeName', 'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr']].head()

In [ ]:
df_all[['Week', 'Stadium', 'Location', 'StadiumType', 'Turf', 'GameWeather',
       'Temperature', 'Humidity', 'WindSpeed', 'WindDirection']].head()

In [ ]:
# 選手一人毎の情報がある列
# 他の列は同一PlayId内では同じ
personal_columms = ['X', 'Y', 'S', 'A', 'Dis', 'Orientation', 'Dir', 
                    'NflId', 'DisplayName', 'JerseyNumber', 
                   'PlayerHeight', 'PlayerWeight', 'PlayerBirthDate','PlayerCollegeName', 'Position']

In [ ]:
df_all.shape

In [ ]:
pd.set_option("display.max_columns", 80)

In [ ]:
df_all.describe()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.tight_layout() 

df_all.hist()
#plt.show()

In [ ]:
df_all.columns

In [ ]:
# ボールを持っている選手の行のみを選択
df_play = df_all[df_all["NflId"]==df_all["NflIdRusher"]]

In [ ]:
df_play.tail()

In [ ]:
df_play["Position"].value_counts()

In [ ]:
df_play[df_play["Position"].isin(["DT", "DE", "CB"])]

In [ ]:
personal_columms

In [ ]:
import seaborn as sns

In [ ]:
sns.jointplot('A', 'Yards', data=df_play)

In [ ]:
sns.jointplot('YardLine', 'Yards', data=df_play[df_play["PossessionTeam"] == df_play["FieldPosition"]])

In [ ]:
sns.jointplot('YardLine', 'Yards', data=df_play[df_play["PossessionTeam"] != df_play["FieldPosition"]])

In [ ]:
sns.jointplot('X', 'Yards', data=df_all[df_all["Position"]=="HB"])

In [ ]:
df_play["Yards"].hist(range=(-20,30),bins=50)

In [ ]:
df_play["Yards"]

In [ ]:
from math import erf

In [ ]:
erf(1)

## ポジションの分類

In [ ]:
df_all["Position"].unique()

In [ ]:
# オフェンスチーム
# QB
# RB（FB、HB、TB）
# WR（SE、FL、SB、WB）
# TE
# OL（C、G、T、E）
# ディフェンスチーム
# DL（DT（NT）、DE）
# LB（ILB（MLB）、OLB（LOLB、ROLB、SLB、WLB））
# DB（CB、S（SS、FS））
# スペシャルチーム
# K、P、LS、H、KR/PR

offence_position = ['WR', 'TE', 'T', 'QB', 'RB', 'G', 'C', 'FB', 'HB',  'OT', 'OG', ]
#OL_position = ['T', 'G', 'C', 'OT', 'OG']
#RB_position = ['RB', 'FB', 'HB']

defence_position = ['SS', 'DE', 'ILB', 'FS', 'CB', 'DT', 'OLB', 'NT', 'MLB', 'LB', 'S', 'DL', 'DB', 'SAF']
#DL_position = ['DL', 'DT', 'DE']
#LB_position = ['LB', 'ILB', ]

In [ ]:
import math

In [ ]:
df_all["offence"] = 0
df_all.loc[df_all["Position"].isin(offence_position), "offence"] = 1

In [ ]:
df_all.head(23)[["Team", "Position", "offence"]]

## ポジション毎の人数

In [ ]:
position_count=df_all.groupby(["PlayId", "Position"]).count()
position_count

In [ ]:
df_position = position_count["GameId"].unstack().fillna(0).astype(int)
df_position

In [ ]:
pd.merge(df_play, df_position, on="PlayId").corr()["Yards"].sort_values()

In [ ]:
df_play

In [ ]:
df_play.loc[:, "OffensePersonnel"] = df_play["OffensePersonnel"].apply(
    lambda x : { i.split(" ")[-1]:int(i.split(" ")[-2]) for i in x.split(",")} )

In [ ]:
df_play.loc[:, "DefensePersonnel"] = df_play["DefensePersonnel"].apply(
    lambda x : { i.split(" ")[-1]:int(i.split(" ")[-2]) for i in x.split(",")} )

In [ ]:
for position in ["DL", "LB", "DB"]:
    df_play.loc[:, position] = [ d[position] for d in df_play["DefensePersonnel"]]
for position in ["RB", "TE", "WR"]:
    df_play.loc[:, position] = [ d[position] for d in df_play["OffensePersonnel"]]    

In [ ]:
df_play

In [ ]:
df_play.corr()["Yards"].sort_values()

In [ ]:
df_play[df_play["Team"]=="home"].groupby("HomeTeamAbbr").mean()["Yards"].sort_values()

In [ ]:
team_home = df_play[df_play["Team"]=="home"].groupby("HomeTeamAbbr")

In [ ]:
df_play["HomeTeamAbbr"].unique()

In [ ]:
team_yards = dict(list(df_play[df_play["Team"]=="home"].groupby("HomeTeamAbbr")["Yards"]))

In [ ]:
plt.hist(team_yards["CAR"], bins=30)
plt.hist(team_yards["WAS"], bins=30)

In [ ]:
dict

In [ ]:
df_play[df_play["Team"]=="home"].groupby("HomeTeamAbbr").mean()["Yards"].hist()

In [ ]:
df_play[df_play["Team"]=="away"].groupby("VisitorTeamAbbr").mean()["Yards"].hist()

In [ ]:
np.array([[1,2,2,5]]).mean()

In [ ]:
df["OffenseFormation"].unique()

In [ ]:
df["OffensePersonnel"].unique()

In [ ]:
df["StadiumType"]

In [ ]:
df["stadiumsype"] = df["StadiumType"].isin()

In [ ]:
df["PlayDirection"].unique()

In [ ]:
personal_yards = df_play[["NflId", "Yards"]].groupby("NflId").agg(["mean", "std", "max", "min", "count"])["Yards"]
personal_yards.dropna(inplace=True)


In [ ]:
personal_yards

In [ ]:
df_play = df_play.merge(personal_yards, on="NflId", how="left")

In [ ]:
df_play.corr()["Yards"].sort_values()

In [ ]:
len(df_play.index)

In [ ]:
df_play["Yards"].value_counts().head(30)

In [ ]:
pd.

In [ ]:
df_play[df_play["Yards"]>50].sort_values("Yards")[["YardLine", "Yards"]]

In [ ]:
len(df_play[df_play["Yards"]+df_play["YardLine"]==100].index)

In [ ]:
df_play[df_play["Yards"]>20].corr()["Yards"].sort_values()

In [ ]:
df_play.loc[:, "over20"] = (df_play["Yards"]>20)*1

In [ ]:
df_play.loc[:, "touchdown"] = (df_play["Yards"]+df_play["YardLine"]==100)

In [ ]:
(df_play["Yards"]+df_play["YardLine"]==100)

In [ ]:
df_play[["over20", "touchdown"]]

In [ ]:
df_play.corr()["touchdown"].sort_values()

In [ ]:
df_play.corr()["over20"].sort_values()